In [327]:
import numpy as np
from numpy.linalg import inv
import math
import itertools
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

time: 1.39 s


In [2]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

/Users/Amiros/anaconda/lib/python3.5/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed autotime.py. To use it, type:
  %load_ext autotime


In [10]:
#load train data as numpy array
X_train = np.genfromtxt('/Users/Amiros/GitHub/Machine Learning for Data Science/hw3/data/gaussian_process/X_train.csv',delimiter=',')
y_train = np.genfromtxt('/Users/Amiros/GitHub/Machine Learning for Data Science/hw3/data/gaussian_process/y_train.csv',delimiter=',')

#load test data as numpy array
X_test = np.genfromtxt('/Users/Amiros/GitHub/Machine Learning for Data Science/hw3/data/gaussian_process/X_test.csv',delimiter=',')
y_test = np.genfromtxt('/Users/Amiros/GitHub/Machine Learning for Data Science/hw3/data/gaussian_process/y_test.csv',delimiter=',')

time: 114 ms


## 1-A

In [440]:
# kernel matrix restricted points in Training data
def K_restricted(X_train, b):
    k = np.linalg.norm(X_train[None,:,:]-X_train[:,None,:],axis=2)
    return np.exp(-1/b * k)

def G_process(X_train, y_train, X_test, b, sigma):
    K_n = K_restricted(X_train, b)
    
    I = np.identity(X_train.shape[0])
    c = np.linalg.inv((sigma) * I + K_n)
    
    m = []
    v = []
    for i in range(X_test.shape[0]):
    # Calculate mean
        distance = np.sqrt(np.sum((X_test[i] - X_train) ** 2, axis=1))
        G_Kernel = np.hstack(np.exp((-1/b) * distance).reshape(1,-1))
        w = np.dot(G_Kernel, c)
        a = np.dot(w, y_train)
        m.append(a)

    return m

time: 22.1 ms


## 1-B

In [441]:
b_list=[5,7,9,11,13,15]
sigma_list=[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]
c = list(itertools.product(b_list, sigma_list))

y_predict = []
for i in range(len(c)):
    y_predict.append(G_process(X_train, y_train, X_test, c[i][0], c[i][1]))
y_predict = np.array(y_predict)

time: 1.95 s


In [442]:
def RMSE(y_predict, y_test, c):
    rmse = []
    
    for i in range(len(c)):
        a = math.sqrt(np.sum((y_predict[i] - y_test)**2) / 42)
        rmse.append(a)  
    return  rmse

time: 4.23 ms


In [443]:
rmse = RMSE(y_predict, y_test, c)
rmse_table = pd.DataFrame(
    {'parameters': c, 'rmse_value': rmse})
rmse_table

,parameters,rmse_value
0,"(5, 0.1)",1.938372
1,"(5, 0.2)",1.933602
2,"(5, 0.3)",1.930739
3,"(5, 0.4)",1.930309
4,"(5, 0.5)",1.931505
5,"(5, 0.6)",1.933778
6,"(5, 0.7)",1.936804
7,"(5, 0.8)",1.940386
8,"(5, 0.9)",1.944405
9,"(5, 1)",1.948782


time: 53 ms


## 1-C

In [444]:
rmse_table.min()

parameters    (5, 0.1)
rmse_value     1.93031
dtype: object

time: 5.46 ms


The best solution is for b= 5 and sigma = 0.1 with rmse value of 1.93031.

This approach comapring to homework 1 gives lower rmse, therefore more accurate result.

## 1-D

In [445]:
# aa = X_train[:,3]
# bb = X_test[:,3]

time: 1.94 ms


# 2